Ref: https://github.com/X-PLUG/mPLUG-Owl/tree/main/mPLUG-Owl3

Run this in vid_env environment. This models works with transformers ver 4.37.2 to 4.47.1

In [1]:
import os 
import torch

In [2]:
print(os.getenv("CONDA_DEFAULT_ENV"))

stable_env


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from transformers import AutoConfig, AutoModel
from transformers import AutoTokenizer, AutoProcessor

In [5]:
from PIL import Image
from decord import VideoReader, cpu

### Model Loading

In [6]:
model_path = 'mPLUG/mPLUG-Owl3-7B-240728'

In [7]:
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_path, 
    torch_dtype=torch.half, 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = model.init_processor(tokenizer)

`torch_dtype` is deprecated! Use `dtype` instead!
HyperQwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


use flash_attn rotary


In [8]:
_ = model.eval().cuda()

In [9]:
model

mPLUGOwl3Model(
  (language_model): HyperQwen2ForCausalLM(
    (model): HyperQwen2Model(
      (embed_tokens): Embedding(151851, 3584)
      (layers): ModuleList(
        (0): HyperQwen2DecoderLayer(
          (self_attn): HyperQwen2SdpaAttention(
            (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
            (rotary_emb_core): RotaryEmbedding()
            (v_kv_proj): Linear(in_features=3584, out_features=1024, bias=True)
            (gate_proj): Sequential(
              (0): Linear(in_features=128, out_features=128, bias=True)
              (1): Sigmoid()
            )
            (v_core_attention_sdpa): ScaleDotProductAttention()
          )
          (mlp): Qwen2MLP(
            

### Prepare Chat Messages

In [ ]:
def create_chat_message(prompt):

    message = [
        {
            "role": "system",
            "content": (
                "You are a vision-language model analyzing lecture videos. "
                "You must rely strictly on visual evidence from the video frames.\n\n"
                "Follow these steps:\n"
                "1. Carefully examine the video frames.\n"
                "2. Perform OCR on any readable text appearing on slides, blackboards, or written material.\n"
                "3. Explicitly list the extracted text before reasoning.\n"
                "4. Answer the question using ONLY the extracted visual information.\n"
                "5. If the required information is not visible or readable, say "
                "\"The answer cannot be determined from the video.\" Do NOT guess.\n"
            )
        },
        {
            "role": "user",
            "content": f"<|video|> {prompt}"
        },
        {
            "role": "assistant",
            "content": ""
        }
    ]
    return message


### Question for Random Test:

From chemistry.mp4

question = 'What was the capacity of the beaker they used in the experiment?'

question = 'What was the melting pointed shown in the experiment?'

question = 'Did they use Potassium chlorate in the experiment?'

question = 'Did they use water in the whole experiment?'

question = 'What are the compounds being used here, list them'

------------------------------------------------------------------------------------------------------

From insertion_sort.mp4

question = 'Which sorting algorithm is being performed in the video?'

question = 'How many element is present in the array shown ?'

question = 'What is the runtime of the algorithm as explained by the professor?'

question = 'Write the elements present in the array the professor is explaining?'

------------------------------------------------------------------------------------------------------

From 15min-video.mp4

question = 'UML is adapted by which group and in which year?'

question = 'How many views of a system are there and what are they?'

### Load Test Video

In [ ]:
videos = ['/home/aritrad/MSR-Project/samples/15min-video.mp4']

In [ ]:
MAX_NUM_FRAMES=32

In [ ]:
def encode_video(video_path):
    
    def uniform_sample(l, n):
        gap = len(l) / n
        idxs = [int(i * gap + gap / 2) for i in range(n)]
        return [l[i] for i in idxs]

    vr = VideoReader(video_path, ctx=cpu(0))
    sample_fps = round(vr.get_avg_fps() / 1)  # FPS
    frame_idx = [i for i in range(0, len(vr), sample_fps)]
    if len(frame_idx) > MAX_NUM_FRAMES:
        frame_idx = uniform_sample(frame_idx, MAX_NUM_FRAMES)
    frames = vr.get_batch(frame_idx).asnumpy()
    frames = [Image.fromarray(v.astype('uint8')) for v in frames]
    #print(frames)
    print('num frames:', len(frames))
    return frames

In [ ]:
video_frames = [encode_video(_) for _ in videos]

In [ ]:
video_frames[0][23]

### Generate Answer

In [ ]:
question = 'How many views of a system are there and what are they?'
message = create_chat_message(question)
inputs = processor(message, images=None, videos=video_frames)

In [ ]:
inputs.to('cuda')
inputs.update({
    'tokenizer': tokenizer,
    'max_new_tokens':512,
    'decode_text':True,
})


In [ ]:
%%time
answer = model.generate(**inputs)
print(answer[0])